In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import sys
#add sarvam_utils path
sys.path.append("../../")
sys.path.append(".")

In [2]:
from sarvam_utils.data.kaggle.spooky_dataset import *

/home/mageswarand/anaconda3/envs/tensorflow1.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
BATCH_SIZE = 32

dataset = TextDataFrame(train_df_path=TRAIN_FILE_PATH,
                        test_df_path=TEST_FILE_PATH,
                        text_col="text",
                        category_col="author",
                        model_name="fast-text-v0")

# To get the features:
train_data = dataset.get_train_data()
val_data = dataset.get_val_data()
test_data = dataset.get_test_data()

# To get indexed category labels:
train_label = dataset.get_train_label()
val_label = dataset.get_val_label()
# test_label = dataset.get_test_label()

#To get on-hot encoded labels:
train_one_hot_encoded_label = dataset.get_train_one_hot_label()
val_one_hot_encoded_label= dataset.get_val_one_hot_label()
# dataset.get_one_hot_test_label()

train_input_fn, train_input_hook = setup_input_graph(train_data,
                                                     train_one_hot_encoded_label,
                                                      batch_size=BATCH_SIZE, 
                                                      scope='train-data')

eval_input_fn, eval_input_hook =  setup_input_graph(dataset.get_val_data(),
                                                     val_one_hot_encoded_label,
                                                    batch_size=1, 
                                                    is_eval=True,
                                                    scope='eval-data')
                                                                                                          
test_input_fn =  test_inputs(dataset.get_test_data(), 
                                        batch_size=1, 
                                        scope='test-data')

Fitting LabelEncoder and LabelBinarizer...
Done!
Splitting the data set(stratified sampling)...
Done!
Preparing vocab file...
Size of train data: 2.332MB
Size of validation data: 0.587MB
Size of test data: 1.240MB
Labels and their document counts based on author
EAP    6320
HPL    4508
MWS    4835
Name: author, dtype: int64
Labels and their document counts based on author
EAP    1580
HPL    1127
MWS    1209
Name: author, dtype: int64
Labels and their document counts based on author
EAP    6320
HPL    4508
MWS    4835
Name: author, dtype: int64
Labels and their document counts based on author
EAP    1580
HPL    1127
MWS    1209
Name: author, dtype: int64
Size of validation data: 0.587MB
Size of test data: 1.240MB


In [4]:
import fast_text_v0

In [5]:
config = fast_text_v0.FastTextConfig(vocab_size=dataset.vocab_count, 
                                     model_dir="fast-text-v0/model/", 
                                     words_vocab_file=dataset.words_vocab_file)

In [6]:
model = fast_text_v0.FastTextV0(config)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'fast-text-v0/model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa7fa440518>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
NUM_EPOCHS = 6
NUM_STEPS = dataset.num_train_samples // BATCH_SIZE
NUM_STEPS

489

In [15]:
# validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
#     input_fn=eval_input_fn,
#     early_stopping_rounds=3,
#     early_stopping_metric="loss",
#     early_stopping_metric_minimize=True,
#     every_n_steps=100)

In [7]:
from sarvam_utils.early_stopping import EarlyStoppingHook

In [9]:
early_stopping_hook = EarlyStoppingHook("reduced_mean:0", 0.30)

INFO:tensorflow:Create EarlyStoppingHook for reduced_mean:0


In [10]:
model.train(input_fn=train_input_fn, hooks=[train_input_hook, early_stopping_hook], steps=NUM_EPOCHS*NUM_STEPS)

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
INFO:tensorflow:table info: <tensorflow.python.ops.lookup_ops.HashTable object at 0x7fa7f8e3efd0>
INFO:tensorflow:words_embed=Tensor("embed-layer/EmbedSequence/embedding_lookup:0", shape=(?, ?, 96), dtype=float32, device=/device:CPU:0)
INFO:tensorflow:words_embed=Tensor("fast_text/Sum:0", shape=(?, 96), dtype=float32, device=/device:GPU:0)
INFO:tensorflow:wide_layer: ------> Tensor("hidden-mlp-layer/dropout_2/dropout/mul:0", shape=(?, 48), dtype=float32)
INFO:tensorflow:logits: ------> Tensor("logits-layer/dense/BiasAdd:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:predicted_class: ------> Tensor("output-layer/class_output:0", shape=(?,), dtype=int64)
INFO:tensorflow:predicted_probabilities: ------> Tensor("output-layer/softmax_output:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:Summary name fast_text/Sum:0 is illegal; using fast_text/Sum_0 instead.
INFO:tensorflow:Summary name output-layer/softmax_output:0 i

INFO:tensorflow:EarlyStoppingHook: Current loss is 1.5662
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.843686
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.844059
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.82694
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.836692
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.864901
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.850172
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.825415
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.813626
INFO:tensorflow:EarlyStoppingHook: Stop at loss

INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.626351
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.843924
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.825598
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.616211
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.574885
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.652281
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 1.07093
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.661272
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.

INFO:tensorflow:EarlyStoppingHook: Current loss is 0.756606
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.577883
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.622789
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.52791
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.469235
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.715702
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.735165
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.898505
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.674084
INFO:tensorflow:EarlyStoppingHook: Stop at lo

INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.435324
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.558127
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.598783
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.570574
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.449914
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.495334
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.531334
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0.580688
INFO:tensorflow:EarlyStoppingHook: Stop at loss 0.3
INFO:tensorflow:EarlyStoppingHook: Current loss is 0

In [11]:
model.evaluate(input_fn=eval_input_fn, hooks=[eval_input_hook])

INFO:tensorflow:table info: <tensorflow.python.ops.lookup_ops.HashTable object at 0x7fa7f061a390>
INFO:tensorflow:words_embed=Tensor("embed-layer/EmbedSequence/embedding_lookup:0", shape=(?, ?, 96), dtype=float32, device=/device:CPU:0)
INFO:tensorflow:words_embed=Tensor("fast_text/Sum:0", shape=(?, 96), dtype=float32, device=/device:GPU:0)
INFO:tensorflow:wide_layer: ------> Tensor("hidden-mlp-layer/dropout_2/Identity:0", shape=(?, 48), dtype=float32)
INFO:tensorflow:logits: ------> Tensor("logits-layer/dense/BiasAdd:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:predicted_class: ------> Tensor("output-layer/class_output:0", shape=(?,), dtype=int64)
INFO:tensorflow:predicted_probabilities: ------> Tensor("output-layer/softmax_output:0", shape=(?, 3), dtype=float32)
INFO:tensorflow:Summary name fast_text/Sum:0 is illegal; using fast_text/Sum_0 instead.
INFO:tensorflow:Summary name output-layer/softmax_output:0 is illegal; using output-layer/softmax_output_0 instead.
INFO:tensorflow:St

{'Accuracy': 0.75229824,
 'Precision': 0.82438433,
 'Recall': 0.87414384,
 'global_step': 291,
 'loss': 0.47339737}